# SOTA-AI Monthly Challenge — Task 2: Kartik’s Misty Problem

This notebook presents an end-to-end solution for **vehicle and pedestrian detection in foggy road scenes** under severe visibility degradation. The core challenge lies in handling **domain shift**: the training dataset consists of clear, sunny-day images, while the test set contains heavily fog-obscured scenes resembling CCTV footage.

To address this, the solution adopts a **domain adaptation strategy via physically inspired fog simulation**, applied offline to the training data. The approach emphasizes:
- Realistic atmospheric degradation (fog density, depth bias, patchiness)
- Preservation of object geometry and class structure
- Recall-oriented inference aligned with the **Detection Quality Index (DQI)** metric

The final pipeline includes:
1. Dataset preparation and fog-based domain adaptation
2. YOLOv8-based object detection training
3. Recall-aware inference and robust submission generation

With the refined fog model and tuned inference strategy, this approach achieved a **DQI score of 0.49504**, placing it at the **top of the leaderboard** at the time of submission.


## 1. Imports and Environment Setup

This section initializes all required libraries and dependencies used throughout the notebook.  
The solution relies on a combination of:

- **Ultralytics YOLOv8** for object detection
- **OpenCV** for image processing and fog simulation
- **NumPy** for numerical operations
- **Pandas** for dataset handling and submission generation
- **Matplotlib** for visual inspection and sanity checks

All experiments were conducted using a GPU-enabled environment (NVIDIA T4 on Google Colab).


In [ ]:
# Installing the necessary libraries

!pip install -U ultralytics opencv-python-headless pandas tqdm pyyaml


In [ ]:
# Importing the necessary libraries

import os, cv2, yaml, random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
from ultralytics import YOLO


In [ ]:
# Setting up the legacy Kaggle API and downloading the dataset

!pip install kaggle

!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

!kaggle competitions download -c kartiks-misty-problem

!unzip kartiks-misty-problem.zip

In [ ]:
# Preparing the dataset directories required for YOLOv8 training

os.makedirs("dataset/images/train_clear", exist_ok=True)
os.makedirs("dataset/images/test", exist_ok=True)
os.makedirs("dataset/labels/train", exist_ok=True)

# copy images
os.system("cp Train/Train/images/* dataset/images/train_clear/")
os.system("cp Test/* dataset/images/test/")


In [ ]:
print(len(os.listdir('/content/dataset/images/train_clear')))

print(len(os.listdir('/content/dataset/images/test')))

## 2. Dataset Preparation via Artificial Fog Simulation

A key challenge in this task is the **domain gap** between clear training images and foggy test images.  
Rather than relying solely on online data augmentation during training, this solution performs **offline fog simulation** to explicitly align the training distribution with the test domain.

### Design Principles of the Fog Model

The fog simulation is designed to closely resemble real-world fog observed in the test set:
- **Global atmospheric haze** reduces overall visibility
- **Depth-aware fog density** increases with distance from the camera
- **Patchy, low-frequency fog variations** mimic uneven fog distribution
- **Mild desaturation** drains color without converting images to grayscale
- **Subtle blur and sensor noise** simulate CCTV-like imaging artifacts

This preprocessing step produces fog-adapted training images while preserving original object labels, enabling robust domain-adaptive learning.


In [ ]:
# Read the training annotations provided as a CSV file.
# Each row corresponds to ONE image, but may contain MULTIPLE objects
# encoded as comma-separated lists (classes and bounding box parameters).
df = pd.read_csv("train.csv")

# Iterate over every image entry in the CSV
for _, row in tqdm(df.iterrows(), total=len(df)):
    # Image identifier (used to name the YOLO label file)
    img_id = str(row["Id"])

    # Parse comma-separated object annotations
    classes = list(map(int, row["Classes"].split(",")))
    xs = list(map(float, row["X_center"].split(",")))
    ys = list(map(float, row["Y_center"].split(",")))
    ws = list(map(float, row["Width"].split(",")))
    hs = list(map(float, row["Height"].split(",")))

    # Create one YOLO-format label file per image.
    # YOLO expects: <class_id> <x_center> <y_center> <width> <height>
    label_path = f"dataset/labels/train/{img_id}.txt"

    with open(label_path, "w") as f:
        # Write one line per object instance in the image
        for c, x, y, w, h in zip(classes, xs, ys, ws, hs):
            f.write(f"{c} {x} {y} {w} {h}\n")


In [ ]:
!head dataset/labels/train/*.txt


In [ ]:
# Selecting a random image from training and test sets to visualize

train_img_dir = "dataset/images/train_clear"
test_img_dir = "dataset/images/test"

clear_img_filename = random.choice(os.listdir(train_img_dir))
clear_img = cv2.imread(os.path.join(train_img_dir, clear_img_filename))
clear_img = cv2.cvtColor(clear_img, cv2.COLOR_BGR2RGB)

test_img_filename = random.choice(os.listdir(test_img_dir))
test_img = cv2.imread(os.path.join(test_img_dir, test_img_filename))
test_img = cv2.cvtColor(test_img, cv2.COLOR_BGR2RGB)

## 3. Fog Transformation Methodology

The fog simulation is implemented as a physically inspired image transformation that approximates atmospheric scattering and visibility loss.

Key components of the transformation include:
- **Depth bias**: Objects farther from the camera are more heavily obscured
- **Exponential transmittance modeling**: Inspired by optical fog models
- **Airlight blending**: Introduces washed-out brightness common in foggy scenes
- **Large-scale fog patches**: Adds spatial non-uniformity
- **Controlled desaturation**: Reduces color intensity without eliminating chromatic information

The parameters were tuned empirically through visual comparison with the test set to achieve a realistic fog impression without over-degrading object structure.


In [ ]:
# Function to simulate fog effect on a clear image and visualizing the results to strike the balance between realism and clarity.

def fog_transform(
    img,
    fog_strength=1.3,      # base fog (global)
    distance_boost=0.55,    # ~10% more fog in distance
    airlight=220,
    desat=0.15,
    blur_ksize=5,
    noise_std=4
):
    img = img.astype(np.float32)
    h, w, _ = img.shape

    # --------------------------------------------------
    # 1. Smooth depth bias (top = far)
    # --------------------------------------------------
    y = np.linspace(1, 0, h)
    depth = np.tile(y[:, None], (1, w))
    depth = depth ** 1.3   # smooth, horizon-heavy

    # --------------------------------------------------
    # 2. Large-scale fog patches (low-frequency)
    # --------------------------------------------------
    fog_noise = np.random.rand(h, w).astype(np.float32)
    fog_noise = cv2.GaussianBlur(fog_noise, (201, 201), 0)
    fog_noise = cv2.normalize(
        fog_noise, None, 0.8, 1.0, cv2.NORM_MINMAX
    )

    # --------------------------------------------------
    # 3. Global fog + gentle distance emphasis
    # --------------------------------------------------
    fog_factor = fog_strength * (1 + distance_boost * depth)
    t = np.exp(-fog_factor * fog_noise)
    t = t[..., None]

    # --------------------------------------------------
    # 4. Atmospheric blending
    # --------------------------------------------------
    A = np.ones_like(img) * airlight
    img = img * t + A * (1 - t)

    # --------------------------------------------------
    # 5. Mild blur (keeps immersion uniform)
    # --------------------------------------------------
    img = cv2.GaussianBlur(img, (blur_ksize, blur_ksize), 0)

    # --------------------------------------------------
    # 6. Soft desaturation
    # --------------------------------------------------
    gray = cv2.cvtColor(img.astype(np.uint8), cv2.COLOR_BGR2GRAY)
    gray = np.stack([gray]*3, axis=-1)
    img = img * (1 - desat) + gray * desat

    # --------------------------------------------------
    # 7. Sensor noise
    # --------------------------------------------------
    img += np.random.normal(0, noise_std, img.shape)

    return np.clip(img, 0, 255).astype(np.uint8)


foggy = fog_transform(clear_img)

plt.figure(figsize=(15,5))
plt.subplot(1,3,1); plt.title("Clear Train"); plt.imshow(clear_img); plt.axis("off")
plt.subplot(1,3,2); plt.title("Fog-Simulated"); plt.imshow(foggy); plt.axis("off")
plt.subplot(1,3,3); plt.title("Real Test"); plt.imshow(test_img); plt.axis("off")
plt.show()

In [ ]:
# Mapping the images in dataset with the fog simulation function to create foggy images for training.

os.makedirs("dataset/images/train_fog", exist_ok=True)

for img_name in tqdm(os.listdir("dataset/images/train_clear")):
    img = cv2.imread(f"dataset/images/train_clear/{img_name}")
    foggy = fog_transform(img)
    cv2.imwrite(f"dataset/images/train_fog/{img_name}", foggy)


In [ ]:
print(len(os.listdir('dataset/images/train_fog')))

In [ ]:
import shutil, os

# Remove broken train dir if it exists
if os.path.exists("dataset/images/train"):
    shutil.rmtree("dataset/images/train")

os.makedirs("dataset/images/train", exist_ok=True)


In [ ]:
# Moving the foggy images in train directory

import shutil, os
from tqdm import tqdm

fog_dir = "dataset/images/train_fog"
train_dir = "dataset/images/train"

for img in tqdm(os.listdir(fog_dir)):
    shutil.copy(
        os.path.join(fog_dir, img),
        os.path.join(train_dir, img)
    )


In [ ]:
# Moving the clear images in train directory with modified names

clear_dir = "dataset/images/train_clear"

for img in tqdm(os.listdir(clear_dir)):
    name, ext = os.path.splitext(img)
    new_name = f"{name}_clear{ext}"
    print(new_name)

    shutil.copy(
        os.path.join(clear_dir, img),
        os.path.join(train_dir, new_name)
    )


In [ ]:
os.makedirs("dataset/images/train", exist_ok=True)

os.system("cp dataset/images/train_clear/* dataset/images/train/")
os.system("cp dataset/images/train_fog/* dataset/images/train/")

In [ ]:
print(len(os.listdir('dataset/images/train')))

## 4. Model Training Strategy

The detection model is based on **YOLOv8 (medium variant)**, selected for its balance between accuracy and computational efficiency under limited GPU resources.

### Training Data Composition
The training set consists of:
- Original **clear images**
- Corresponding **fog-simulated images**

This mixed-domain strategy ensures:
- Clear images reinforce object geometry and class identity
- Foggy images promote robustness to low-visibility conditions

### Training Configuration Highlights
- Optimizer: AdamW
- Image size: 640 × 640
- Mixed-domain training (clear + fog)
- Geometric augmentations only (no additional photometric distortion)
- Moderate number of epochs to avoid overfitting

The training objective is **not raw mAP maximization**, but stable localization and recall under fog, aligned with the DQI metric.


In [ ]:
# Creating the data.yaml file required for YOLOv8 training

data_yaml = {
    "path": "dataset",
    "train": "images/train",
    "val": "images/train",
    "names": {
        0:"Truck",1:"Cyclist",2:"Biker",3:"Mini Truck",4:"Car 1",
        5:"Jeep",6:"Toto",7:"Carrier Motor-Rikshaw",8:"Auto Rikshaw",
        9:"Bus",10:"Tempo",11:"Pedal Rikshaw",12:"Pedestrian",
        13:"Car 2",14:"Tractor"
    }
}

with open("data.yaml","w") as f:
    yaml.safe_dump(data_yaml, f, sort_keys=False)


In [ ]:
# Load a YOLOv8 Medium model pretrained on COCO.
# We choose yolov8m as a balance between:
# - strong localization capacity (important for DQI)
# - feasible training time on limited GPU resources
model = YOLO("yolov8m.pt")

model.train(
    data="data.yaml",      # dataset definition (paths + class names)
    epochs=15,             # moderate epochs to avoid overfitting
    imgsz=640,             # standard resolution for detection tasks
    batch=8,               # tuned for Colab T4 memory limits

    # Optimizer choice:
    # AdamW provides more stable convergence under heavy domain shift
    optimizer="AdamW",
    lr0=5e-4,              # low initial LR to preserve pretrained features
    cos_lr=True,           # cosine decay for smooth convergence
    warmup_epochs=2,       # prevents early training instability
    patience=0,            # disable early stopping (fixed budget training)

    # Geometric augmentations only:
    # Photometric effects are handled OFFLINE via fog simulation
    mosaic=1.0,
    scale=0.4,
    translate=0.1,
    fliplr=0.5,

    cache=True,            # speeds up training by caching images
    workers=2,             # conservative parallelism for Colab
    device=0,              # GPU
    plots=True,            # save training curves for transparency
    verbose=True
)


## 5. Inference Strategy

Inference is configured with a **recall-first philosophy**, reflecting the structure of the Detection Quality Index (DQI), which penalizes missed detections more severely than loose bounding boxes.

Key inference choices include:
- **Lower confidence threshold (conf = 0.12)** to reduce false negatives
- Conservative non-maximum suppression
- Filtering of extremely small boxes to avoid noise

This setup significantly reduces the number of test images with zero detections, directly improving per-class F1 scores and overall DQI.


In [ ]:
# results = model.predict(
#     source="dataset/images/test",
#     imgsz=640,
#     conf=0.15,     # recall-biased
#     iou=0.65,
#     max_det=300,
#     device=0
# )

from ultralytics import YOLO

# Load the best-performing model checkpoint from training
model = YOLO("/content/runs/detect/train/weights/best.pt")

# Run inference in STREAMING mode to reduce memory usage.
# A lower confidence threshold is used to prioritize recall,
# which is critical for maximizing DQI under heavy fog.
results_gen = model.predict(
    source="/content/Test",
    imgsz=640,
    conf=0.12,             # recall-first threshold
    iou=0.65,              # balanced NMS to avoid duplicate boxes
    max_det=300,           # allow dense scenes
    batch=1,               # stable inference under fog
    stream=True,           # generator-based inference
    save=True,             # save visual predictions for inspection
)

# Materialize the generator.
# This is necessary because Python generators are consumed after one pass.
results = []
for r in results_gen:
    results.append(r)




## 6. Submission CSV Preparation

The competition requires predictions to be aggregated **per image**, with all detected objects represented as comma-separated values in a single row.

This section:
- Aggregates YOLO predictions image-wise
- Ensures all test images are included in the submission
- Handles edge cases where no detections are produced
- Formats outputs strictly according to the required schema:
  - Id
  - Classes
  - X_center
  - Y_center
  - Width
  - Height

Special care is taken to ensure that the submission contains **exactly one row per test image**, avoiding invalid or incomplete entries.


In [ ]:
from collections import defaultdict
import os
import pandas as pd

# Dictionary to aggregate predictions PER IMAGE.
# The competition expects all detections for one image in a single row.
records = defaultdict(lambda: {
    "Classes": [],
    "X_center": [],
    "Y_center": [],
    "Width": [],
    "Height": []
})

# Iterate only over YOLO results (not filenames),
# since each result object already contains its image path.
for r in results:
    # Extract image ID from the file path
    img_path = r.path
    img_id = os.path.splitext(os.path.basename(img_path))[0]

    # Skip images with no detections
    if r.boxes is None or len(r.boxes) == 0:
        continue

    for b in r.boxes:
        # Extract normalized YOLO bounding box (x, y, w, h)
        x, y, w, h = b.xywhn[0].tolist()

        # Filter extremely tiny boxes to avoid noise
        if w < 0.005 or h < 0.005:
            continue

        # Append predictions in comma-separated format
        records[img_id]["Classes"].append(str(int(b.cls.item())))
        records[img_id]["X_center"].append(f"{x:.6f}")
        records[img_id]["Y_center"].append(f"{y:.6f}")
        records[img_id]["Width"].append(f"{w:.6f}")
        records[img_id]["Height"].append(f"{h:.6f}")


In [ ]:
import os
import pandas as pd

# Fallback detection used ONLY when the model predicts nothing for an image.
# A single, conservative central vehicle minimizes DQI damage compared to
# missing rows or random hallucinations.
FALLBACK = {
    "Classes": "4",          # Car 1 (most common & safest class)
    "X_center": "0.500000",
    "Y_center": "0.550000",
    "Width": "0.300000",
    "Height": "0.200000"
}

# Ensure every test image appears exactly once in the submission
test_images = sorted(os.listdir("/content/Test"))
rows = []

for img_name in test_images:
    img_id = os.path.splitext(img_name)[0]

    if img_id in records and len(records[img_id]["Classes"]) > 0:
        # Normal case: model produced detections
        v = records[img_id]
        rows.append({
            "Id": img_id,
            "Classes": ",".join(v["Classes"]),
            "X_center": ",".join(v["X_center"]),
            "Y_center": ",".join(v["Y_center"]),
            "Width": ",".join(v["Width"]),
            "Height": ",".join(v["Height"])
        })
    else:
        # Defensive fallback for no-detection cases
        rows.append({
            "Id": img_id,
            **FALLBACK
        })

# Write final submission file
submission = pd.DataFrame(rows)
submission.to_csv("task2_final_submission.csv", index=False)

print("Rows in submission:", len(submission))


In [ ]:
print(len(os.listdir('/content/runs/detect/predict')))

## 7. Qualitative Evaluation on Test Images

Before final submission, qualitative inspection is performed on randomly selected test images to verify:
- Correct bounding box placement
- Reasonable object localization under fog
- Detection of small and distant objects
- Class consistency (e.g., pedestrians vs. vehicles)

Visual inspection complements quantitative metrics and helps identify failure modes such as:
- Missed detections in dense fog
- Overly large or misplaced bounding boxes

This step ensures the model’s behavior aligns with expectations and provides confidence in the final submission.


In [ ]:
# Visualzing some random predictions from the submission file to understand the model performance

import random
import cv2
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

# Class names (adjust if needed)
CLASS_NAMES = {
    0:"Truck", 1:"Cyclist", 2:"Biker", 3:"Mini Truck", 4:"Car 1",
    5:"Jeep", 6:"Toto", 7:"Carrier MR", 8:"Auto Rikshaw",
    9:"Bus", 10:"Tempo", 11:"Pedal Rikshaw",
    12:"Pedestrian", 13:"Car 2", 14:"Tractor"
}

# Fixed color palette for consistency
np.random.seed(42)
CLASS_COLORS = {
    k: tuple(np.random.randint(0,255,3).tolist())
    for k in CLASS_NAMES
}


def visualize_row(row, img_dir="/content/Test"):
    img_path = f"{img_dir}/{row['Id']}.jpg"
    img = cv2.imread(img_path)

    if img is None:
        print(f"⚠️ Image not found: {img_path}")
        return

    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    h, w, _ = img.shape

    # Handle empty predictions safely
    if pd.isna(row["Classes"]) or row["Classes"] == "":
        plt.figure(figsize=(7,4))
        plt.imshow(img)
        plt.title(f"{row['Id']}  |  NO PREDICTIONS")
        plt.axis("off")
        plt.show()
        return

    classes = list(map(int, row["Classes"].split(",")))
    xs = list(map(float, row["X_center"].split(",")))
    ys = list(map(float, row["Y_center"].split(",")))
    ws = list(map(float, row["Width"].split(",")))
    hs = list(map(float, row["Height"].split(",")))

    for c, x, y, bw, bh in zip(classes, xs, ys, ws, hs):
        x1 = int((x - bw/2) * w)
        y1 = int((y - bh/2) * h)
        x2 = int((x + bw/2) * w)
        y2 = int((y + bh/2) * h)

        color = CLASS_COLORS.get(c, (255,255,255))
        label = CLASS_NAMES.get(c, str(c))

        cv2.rectangle(img, (x1,y1), (x2,y2), color, 2)
        cv2.putText(
            img,
            label,
            (x1, max(y1-6, 15)),
            cv2.FONT_HERSHEY_SIMPLEX,
            0.6,
            color,
            2
        )

    plt.figure(figsize=(8,5))
    plt.imshow(img)
    plt.title(f"{row['Id']}  |  {len(classes)} detections")
    plt.axis("off")
    plt.show()


df = pd.read_csv("task2_final_submission.csv")

sampled_rows = df.sample(5, random_state=random.randint(0, 10_000))

for _, row in sampled_rows.iterrows():
    visualize_row(row)


In [ ]:
from google.colab import files

files.download('task2_final_submission.csv')


In [ ]:
files.download('/content/runs/detect/train/weights/best.pt')

## Final Remarks

This notebook demonstrates a practical, competition-oriented approach to handling extreme domain shift in object detection tasks. By combining physics-inspired fog simulation, careful dataset construction, and metric-aware inference, the solution achieves strong performance under challenging conditions.

The methodology emphasizes:
- Transparency
- Reproducibility
- Engineering discipline over brute-force computation

Further improvements may be explored through fine-grained per-class inference tuning or additional domain-specific augmentations.

Thank you for this challenge SOTA-AI Community.
